# Model logitowy

(Estymacja regresji logistycznej)

#### Dane: 
- choroby serca 
- z Kaggle

#### Zmienna objaśniana: 
- CHD - Congenital Heart Disease/Defect
- TenYearCHD - CHD w ciągu 10 lat

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
import os
import progressbar
# py -m pip install progressbar2
# py -m pip install openpyxl
# py -m pip install statsmodels
# py -m pip install seaborn
import openpyxl as xl

In [4]:
### wczytanie danych

mydata = pd.read_csv('framingham.csv')

# zmieniamy nazwę zmiennej objaśnianej z 'TenYearCHD' na 'Dependent_CHD' (jest to ostatnia zmienna w tym zbiorze danych)
mydata.rename(columns = {'TenYearCHD':'Dependent_CHD'}, inplace = True)
mydata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  Dependent_CHD    4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


### Pozbycie się missing values

In [12]:
mydata.isnull().sum().sort_values(ascending = False)

### usuwamy rekordy w pełni puste
mydata.dropna(how = 'all', inplace = True)
mydata.isnull().sum().sort_values(ascending = False)
# wniosek --> nie ma pustych rekordów, wszystkie są przynajmniej częściowo pełne

### usuwamy niepełne rekordy ze zbioru:

# 1) lokalizujemy indeksy takich rekordów
ids_with_nan = mydata[mydata.isna().any(axis = 1)]
print('Liczba rekordów częściowo pustych:', ids_with_nan.shape[0])

# 2) usuwamy takie rekordy
mydata_f = mydata.dropna()
mydata_f.info()

Liczba rekordów częściowo pustych: 582
<class 'pandas.core.frame.DataFrame'>
Index: 3656 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3656 non-null   int64  
 1   age              3656 non-null   int64  
 2   education        3656 non-null   float64
 3   currentSmoker    3656 non-null   int64  
 4   cigsPerDay       3656 non-null   float64
 5   BPMeds           3656 non-null   float64
 6   prevalentStroke  3656 non-null   int64  
 7   prevalentHyp     3656 non-null   int64  
 8   diabetes         3656 non-null   int64  
 9   totChol          3656 non-null   float64
 10  sysBP            3656 non-null   float64
 11  diaBP            3656 non-null   float64
 12  BMI              3656 non-null   float64
 13  heartRate        3656 non-null   float64
 14  glucose          3656 non-null   float64
 15  Dependent_CHD    3656 non-null   int64  
dtypes: float64(9), int64(7)
me

### Transformacja zmiennej BMI
wartości optymalne BMI to: 18.5 - 24.9